# PatchCore Transformed

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import

In [64]:
import importlib
importlib.reload(Thesis_dev)

import os
import Thesis_dev
from torch.utils.data import DataLoader
from Thesis_dev import MVTecDataset, PatchCoreViT


In [65]:
folder_name = "/content/drive/MyDrive/bottle_xs"

train_path, test_path = os.path.join(folder_name, "train", "good"), os.path.join(folder_name, "test")
train_paths, test_paths = [train_path], [ os.path.join(test_path, anomaly) for anomaly in os.listdir(test_path) if anomaly[0] != '.']

In [66]:
backbones = [
    "timm/wide_resnet50_2.tv_in1k",
    "google/vit-base-patch16-224-in21k"
]

# Config
backbone = "google/vit-base-patch16-224-in21k"
layers = [4]

# Training
pcViT = PatchCoreViT(layers=layers, backbone=backbone, f_coreset=1)
pcViT.fit(train_paths)

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


[INFO][__init__] Model PatchCore loaded on device: cpu


100%|██████████| 5/5 [00:04<00:00,  1.14it/s]


In [67]:
# Testing
pcViT.evaluate(test_paths, validation_flag = True)


100%|██████████| 10/10 [00:09<00:00,  1.02it/s]


Val: Image Level ROCAUC: 1.0
Val: Pixel Level ROCAUC: 0.9517494227406629
[INFO][evaluate] Image Level ROCAUC: 1.000
[INFO][evaluate] Initial Score Threshold: 4.773 F1Score: 0.889
[INFO][evaluate] Optimal Score Threshold: 3.313 F1Score: 1.000
[INFO][evaluate] Average Inference time with batch_size=1: 0.917s


## Testing

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_dataloader = pcViT.get_dataloader([train_path])
test_dataloader = pcViT.get_dataloader(test_paths)
for sample, target, filepath, mask in train_dataloader:

            sample_preprocessed = sample.pixel_values[0].to(device)
            patch, _ = pcViT.extract_embeddings(sample_preprocessed)
            break

In [ ]:
temp, feature_maps = pcViT.extract_embeddings(sample_preprocessed)

In [ ]:
temp = pcViT(sample_preprocessed)

In [ ]:
print(test_dataloader.dataset.filepaths[0])
print(test_dataloader.dataset.filepaths[0].replace('test','ground_truth'))
print(test_dataloader.dataset.filepaths[0].replace('.','_mask.'))

/content/drive/MyDrive/bottle_xs/test/broken_large/004.png
/content/drive/MyDrive/bottle_xs/ground_truth/broken_large/004.png
/content/drive/MyDrive/bottle_xs/test/broken_large/004_mask.png


In [78]:
temp1, temp2 = pcViT.knn_dists.topk(k=5, largest=False)
temp1.shape

torch.Size([1, 5])

In [90]:
pcViT.cm

array([[5, 0],
       [0, 5]])

In [68]:
temp1, temp2 = pcViT.distances.topk(k=5, largest=False)
temp1

tensor([[0.5389, 0.5954, 0.6269, 0.6552, 0.8571],
        [0.3825, 0.4889, 0.4963, 0.5724, 0.7438],
        [0.2346, 0.2754, 0.3195, 0.3255, 0.4346],
        [0.5741, 0.7065, 0.7296, 0.8809, 2.0768],
        [0.8423, 0.9219, 0.9559, 1.1056, 2.1536],
        [1.0768, 1.2236, 1.2523, 1.4430, 3.3760],
        [1.3101, 1.3282, 1.3416, 1.4377, 2.3236],
        [1.5019, 1.5185, 1.5257, 1.6080, 2.2904],
        [1.3496, 1.3842, 1.5009, 1.9523, 3.0283],
        [0.9600, 1.0344, 1.0946, 2.2220, 2.2367],
        [0.7074, 0.8716, 0.8957, 0.9290, 1.2830],
        [0.4721, 0.6176, 0.6546, 0.6619, 0.9545],
        [0.4270, 0.5340, 0.5933, 0.6235, 0.6948],
        [0.5159, 0.6345, 0.6474, 0.6588, 0.8964],
        [0.5391, 0.6049, 0.6074, 0.7109, 0.8381],
        [0.4770, 0.5460, 0.5575, 0.6899, 0.8236],
        [0.9169, 1.0233, 1.1641, 1.1769, 1.8577],
        [0.9688, 1.3243, 1.4143, 1.5970, 1.9726],
        [1.1213, 1.6887, 1.9726, 2.1410, 2.4179],
        [1.7866, 2.4642, 2.6975, 2.7315, 2.9515],
